In [1]:
from py.Training import training_and_testing, loss_function
from py.vae import NeuralNet as nnet
import py.config as CFG

In [2]:
import os
import glob
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchinfo import summary

from matplotlib import pyplot as plt
plt.rc('font', family='NanumGothic')
from PIL import Image

In [3]:
def rotate_images(image):
    # 이미지를 0도, 90도, 180도, 270도로 회전시키는 함수
    images = []
    for angle in [0, 90, 180, 270]:
        rotated_image = TF.rotate(image, angle)
        images.append(rotated_image)
    return images

def gauss_noise(image_tensor, sigma=0.05):
    # 이미지에 가우시안 노이즈를 추가하는 함수
    noise = torch.randn(image_tensor.size()) * sigma
    noisy_image = image_tensor + noise
    noisy_image = torch.clamp(noisy_image, 0, 1)
    return noisy_image


class VAECustomDataset(Dataset):
    def __init__(self, file_paths, transform=None, gauss_sigma=0.05):
        # 데이터셋 초기화
        self.file_paths = file_paths
        self.transform = transform
        self.gauss_sigma = gauss_sigma

    def __len__(self):
        # 데이터셋의 길이를 반환
        return len(self.file_paths)

    def __getitem__(self, idx):
        # 주어진 인덱스에 해당하는 데이터를 반환
        image = Image.open(self.file_paths[idx])
        if self.transform:
            # 변환이 주어진 경우
            original_image = self.transform(image)  # 원본 이미지 변환
            noisy_image = gauss_noise(original_image, self.gauss_sigma)  # 노이즈 추가 이미지 생성
            images = rotate_images(image)  # 회전 이미지 생성
            transformed_images = [self.transform(img) for img in images]  # 회전 이미지를 변환
            noisy_images = [gauss_noise(img, self.gauss_sigma) for img in transformed_images]  # 회전된 이미지에 노이즈 추가
            return original_image, noisy_image, transformed_images, noisy_images
        else:
            # 변환이 주어지지 않은 경우 원본 이미지 반환
            return image


In [4]:
trainset = torchvision.datasets.CIFAR10(root='D:\PPJ\Model\pre_data', train=True, download=True,
                                        transform=CFG.transform_pre)
train_loader = DataLoader(dataset=trainset,batch_size=CFG.batch_size, shuffle=True,
                          num_workers=2)

Files already downloaded and verified


In [5]:
# 데이터 폴더 경로 설정
folder = r"D:\PPJ\Model\data\Normal"
file_path = glob.glob(os.path.join(folder, "*.jpg"))

In [6]:
train_size = int(0.8 * len(file_path))
test_size = len(file_path) - train_size

train_paths, test_paths = random_split(file_path, [train_size, test_size])

train_dataset = VAECustomDataset(file_paths=train_paths, transform=CFG.transform)
test_dataset = VAECustomDataset(file_paths=test_paths, transform=CFG.transform)

In [7]:
neuralnet = nnet(height=CFG.height, width=CFG.width, channel=CFG.channel,
                 device=CFG.device, ngpu=CFG.ngpu, ksize=CFG.ksize, z_dim=CFG.z_dim)

The number of parameters: 968903145


In [8]:
summary(neuralnet.encoder, input_size=(32, 1, 480, 480), device='cpu')

torch.Size([32, 20])


Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [32, 20]                  --
├─Sequential: 1-1                        [32, 64, 123, 123]        --
│    └─Conv2d: 2-1                       [32, 16, 481, 481]        272
│    └─ELU: 2-2                          [32, 16, 481, 481]        --
│    └─Conv2d: 2-3                       [32, 16, 482, 482]        4,112
│    └─ELU: 2-4                          [32, 16, 482, 482]        --
│    └─MaxPool2d: 2-5                    [32, 16, 241, 241]        --
│    └─Conv2d: 2-6                       [32, 32, 242, 242]        8,224
│    └─ELU: 2-7                          [32, 32, 242, 242]        --
│    └─Conv2d: 2-8                       [32, 32, 243, 243]        16,416
│    └─ELU: 2-9                          [32, 32, 243, 243]        --
│    └─MaxPool2d: 2-10                   [32, 32, 121, 121]        --
│    └─Conv2d: 2-11                      [32, 64, 122, 122]        32,832


In [19]:
summary(neuralnet.decoder, input_size=(32, 20), device=CFG.device)

Layer (type:depth-idx)                   Output Shape              Param #
Decoder                                  [32, 1, 495, 495]         --
├─Sequential: 1-1                        [32, 921600]              --
│    └─Linear: 2-1                       [32, 512]                 10,752
│    └─ELU: 2-2                          [32, 512]                 --
│    └─Linear: 2-3                       [32, 921600]              472,780,800
│    └─ELU: 2-4                          [32, 921600]              --
├─Sequential: 1-2                        [32, 1, 495, 495]         --
│    └─Conv2d: 2-5                       [32, 64, 121, 121]        65,600
│    └─ELU: 2-6                          [32, 64, 121, 121]        --
│    └─Conv2d: 2-7                       [32, 64, 122, 122]        65,600
│    └─ELU: 2-8                          [32, 64, 122, 122]        --
│    └─ConvTranspose2d: 2-9              [32, 32, 245, 245]        51,232
│    └─ELU: 2-10                         [32, 32, 245, 245] 

In [ ]:
def pretrain(neuralnet, train_loader, epochs):
    neuralnet.train(mode=True)
    for epoch in tqdm(range(epochs)):
        for i, (inputs, _) in enumerate(train_loader):
            input = inputs.to(device=CFG.device)
            enc = mu, sigma = neuralnet.encoder(input)
            x_hat = neuralnet.decoder(enc)
            loss = loss_function(
                x=input, x_hat=x_hat, mu=mu, sigma=sigma
            )
            neuralnet.optimizer.zero_grad()
            loss.backward()
            neuralnet.optimizer.step()
            
            if i % 10 == 0:
                print(f'Epoch : [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    torch.save(neuralnet.state_dict(), 'vae_cifar10_pretrained.pth')

In [ ]:
pretrain(neuralnet=neuralnet, train_loader=train_loader, epochs=CFG.epochs)

  0%|          | 0/100 [00:14<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
training_and_testing(neuralnet=neuralnet ,train_dataset=train_dataset, test_dataset= test_dataset,
                      epochs=CFG.epochs, batch_size=CFG.batch_size)

AttributeError: 'NeuralNet' object has no attribute 'train'